# Open in Colab Badge and Runtime Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/piumalnipun9/Inventory-Management-Forecasting-using-Machine-Learning/blob/main/notebooks/colab_run.ipynb)

This notebook lets you edit locally in VS Code while executing on a Google Colab runtime via a secure tunnel, and also provides robust loading/cleaning for `Grocery_Inventory_new_v1.csv`.

COLAB = False
try:
    import google.colab  # type: ignore
    COLAB = True
except Exception:
    COLAB = False
print(f"Running in Colab: {COLAB}")


In [ ]:
# Install and Import Dependencies

If running on Colab, we'll install required packages. Locally in VS Code, you can skip this cell and use your own environment.

import sys, os, subprocess, textwrap, random, string, json, time, io, re
from datetime import datetime, timedelta

# Minimal set here; the repo's requirements.txt will be installed if in Colab.
if 'google.colab' in sys.modules:
    %pip -q install --upgrade pip
    %pip -q install pandas pyarrow numpy seaborn matplotlib cloudflared jinja2
    # Also install project requirements if present
    REPO_URL = 'https://github.com/piumalnipun9/Inventory-Management-Forecasting-using-Machine-Learning.git'
    WORKDIR = '/content/Inventory-Management-Forecasting-using-Machine-Learning'
    if not os.path.exists(WORKDIR):
        !git clone --depth=1 $REPO_URL $WORKDIR
    else:
        %cd $WORKDIR
        !git pull --rebase --autostash
        %cd -
    REQ = os.path.join(WORKDIR, 'requirements.txt')
    if os.path.exists(REQ):
        %pip -q install -r $REQ

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 4)
print('Dependencies ready')


In [ ]:
# Google Drive Auth/Mount (Colab) and Local/Upload Fallback

If on Colab, we can mount Drive and also support file upload for `Grocery_Inventory_new_v1.csv`.

COLAB = 'google.colab' in sys.modules
UPLOADS = {}
DRIVE_MOUNT = '/content/drive'
if COLAB:
    try:
        from google.colab import drive, files  # type: ignore
        drive.mount(DRIVE_MOUNT, force_remount=False)
    except Exception as e:
        print('Drive mount skipped:', e)
    try:
        print('If your CSV is not on Drive, use the uploader below to select it from your computer...')
        UPLOADS = files.upload()  # opens a dialog in Colab UI
        print('Uploaded files:', list(UPLOADS.keys()))
    except Exception as e:
        print('Upload dialog not available or canceled:', e)
else:
    print('Running outside Colab; Drive mount and uploader are skipped.')


In [ ]:
# Robust CSV Loader for Grocery_Inventory_new_v1.csv

This will try these paths in order: local Windows path, Drive path, or uploaded file.

from pathlib import Path

def load_csv() -> Path | None:
    candidates = []
    # 1) Local Windows path as provided in VS Code workspace
    candidates.append(Path(r"e:/Web Development/Inventory_Manager/Grocery_Inventory_new_v1.csv"))
    candidates.append(Path(r"E:/Web Development/Inventory_Manager/Grocery_Inventory_new_v1.csv"))
    # 2) If running in Colab and Drive is mounted, look under MyDrive common locations
    if COLAB:
        drive_candidates = [
            Path(DRIVE_MOUNT) / 'MyDrive' / 'Grocery_Inventory_new_v1.csv',
            Path(DRIVE_MOUNT) / 'MyDrive' / 'Inventory' / 'Grocery_Inventory_new_v1.csv',
        ]
        candidates.extend(drive_candidates)
    # 3) Uploaded file in Colab
    if COLAB and UPLOADS:
        for name in UPLOADS.keys():
            if name.lower() == 'grocery_inventory_new_v1.csv':
                # Save uploaded content to the current working directory
                open(name, 'wb').write(UPLOADS[name])
                candidates.insert(0, Path(name))
                break
    for p in candidates:
        try:
            if p.exists():
                # Probe readable
                _df = pd.read_csv(p, engine='python', dtype=str, nrows=5)
                print('Found CSV at:', p)
                return p
        except Exception:
            continue
    print('CSV not found. Please ensure the file exists in one of the probed paths or upload via the widget.')
    return None

CSV_PATH = load_csv()


In [ ]:
# Data Cleaning Utilities

# Helpers to normalize and parse fields

import math

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c: c.strip() for c in df.columns}
    df = df.rename(columns=cols)
    if 'Catagory' in df.columns and 'Category' not in df.columns:
        df = df.rename(columns={'Catagory': 'Category'})
    return df

def to_money(x) -> float:
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return 0.0
    s = str(x).strip().replace('$','').replace(',','')
    try:
        return float(s)
    except Exception:
        return 0.0

def to_percent(x) -> float:
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return 0.0
    s = str(x).strip().replace('%','')
    # handle cases like '089', '-161', '001'
    s = re.sub(r'[^0-9\-\.]+', '', s)
    if s == '' or s == '-' or s == '.':
        return 0.0
    try:
        v = float(s) / 100.0
        # sanity clip to [-3, 3]
        return max(-3.0, min(3.0, v))
    except Exception:
        return 0.0

def to_int_safe(x) -> int:
    try:
        return int(float(str(x).strip()))
    except Exception:
        return 0

def to_date_safe(x):
    if x is None:
        return pd.NaT
    s = str(x).strip()
    for fmt in ['%m/%d/%Y', '%Y-%m-%d', '%d/%m/%Y']:
        try:
            return pd.to_datetime(s, format=fmt, errors='raise')
        except Exception:
            pass
    return pd.to_datetime(s, errors='coerce')

raw_df = None
if CSV_PATH is not None:
    raw_df = pd.read_csv(CSV_PATH, engine='python', dtype=str)
    raw_df = rename_columns(raw_df)
    # Parse and normalize key columns
    for col in ['Unit_Price']:
        if col in raw_df.columns:
            raw_df[col] = raw_df[col].map(to_money)
    if 'percentage' in raw_df.columns:
        raw_df['percentage'] = raw_df['percentage'].map(to_percent)
    for col in ['Stock_Quantity','Reorder_Level','Reorder_Quantity','Sales_Volume','Inventory_Turnover_Rate']:
        if col in raw_df.columns:
            raw_df[col] = raw_df[col].map(to_int_safe)
    for col in ['Date_Received','Last_Order_Date','Expiration_Date']:
        if col in raw_df.columns:
            raw_df[col] = raw_df[col].map(to_date_safe)

print('Cleaning complete' if raw_df is not None else 'No CSV loaded')


In [ ]:
# Data Validation and Fixups

changes = {}
if raw_df is not None:
    df = raw_df.copy()
    # Fill missing Category from Product_Name heuristics
    if 'Category' in df.columns and 'Product_Name' in df.columns:
        mask_blank = df['Category'].isna() | (df['Category'].astype(str).str.strip() == '')
        inferred = df.loc[mask_blank, 'Product_Name'].astype(str).str.lower().map({
            'milk': 'Dairy', 'cheese': 'Dairy', 'yogurt': 'Dairy',
            'apple': 'Produce', 'banana': 'Produce', 'carrot': 'Produce',
            'bread': 'Bakery', 'cake': 'Bakery'
        }).fillna('Grocery')
        before = mask_blank.sum()
        df.loc[mask_blank, 'Category'] = inferred
        after = df['Category'].isna().sum()
        changes['filled_category'] = int(before - after)
    # Clip percentage
    if 'percentage' in df.columns:
        before = df['percentage'].copy()
        df['percentage'] = df['percentage'].clip(-3.0, 3.0)
        changes['percentage_clipped'] = int((before != df['percentage']).sum())
    # Drop negative quantities
    for col in ['Stock_Quantity','Reorder_Level','Reorder_Quantity','Sales_Volume']:
        if col in df.columns:
            neg = (df[col] < 0).sum()
            if neg > 0:
                df.loc[df[col] < 0, col] = 0
                changes[f'neg_{col}_to_zero'] = int(neg)
    # Normalize Status
    if 'Status' in df.columns:
        mapping = {'active':'Active','backordered':'Backordered','discontinued':'Discontinued'}
        df['Status'] = df['Status'].astype(str).str.strip().str.lower().map(mapping).fillna('Active')
        changes['status_normalized'] = int(len(df))
else:
    df = None

print('Validation complete. Changes:', changes)


In [ ]:
# Quick EDA Checks and Queries

if df is not None:
    display(df.info())
    display(df.head(10))
    if 'Stock_Quantity' in df.columns:
        display(df.sort_values('Stock_Quantity', ascending=False).head(10))
    if set(['Stock_Quantity','Reorder_Level']).issubset(df.columns):
        low = df[df['Stock_Quantity'] <= df['Reorder_Level']]
        print('Items at/below reorder level:', len(low))
        display(low.head(10))
    if 'Expiration_Date' in df.columns:
        soon = df[df['Expiration_Date'] <= (pd.Timestamp.today() + pd.Timedelta(days=30))]
        print('Expiring within 30 days:', len(soon))
        display(soon.head(10))
    if set(['Category','Unit_Price']).issubset(df.columns):
        display(df.groupby('Category')['Unit_Price'].mean().sort_values(ascending=False).head(10))
    # Optional simple plot
    if set(['Category','Stock_Quantity']).issubset(df.columns):
        sns.barplot(x='Category', y='Stock_Quantity', data=df)
        plt.xticks(rotation=45, ha='right')
        plt.title('Stock by Category')
        plt.show()
else:
    print('No data loaded for EDA')


In [ ]:
# Save Cleaned Data (CSV/Parquet)

output_dir = 'cleaned'
if df is not None:
    os.makedirs(output_dir, exist_ok=True)
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_out = os.path.join(output_dir, f'Grocery_Inventory_cleaned_{ts}.csv')
    pq_out = os.path.join(output_dir, f'Grocery_Inventory_cleaned_{ts}.parquet')
    df.to_csv(csv_out, index=False)
    try:
        df.to_parquet(pq_out, index=False)
        print('Saved:', csv_out, 'and', pq_out)
    except Exception as e:
        print('Parquet save skipped:', e)
else:
    print('No cleaned data to save')


In [ ]:
# Run Project Conversion and Pipeline in Colab

if 'google.colab' in sys.modules:
    WORKDIR = '/content/Inventory-Management-Forecasting-using-Machine-Learning'
    %cd $WORKDIR

# Write the uploaded/loaded CSV to repo root if not already
if CSV_PATH is not None:
    target = Path('Grocery_Inventory_new_v1.csv')
    if not target.exists() or Path(CSV_PATH).resolve() != target.resolve():
        try:
            tmp_df = pd.read_csv(CSV_PATH, engine='python')
            tmp_df.to_csv(target, index=False)
            print('Copied CSV into repo root for scripts:', target)
        except Exception as e:
            print('Copy skipped:', e)
else:
    print('CSV not available; you can still run with synthetic data by setting --generate-synthetic')

# Conversion
if Path('Grocery_Inventory_new_v1.csv').exists():
    !python -m scripts.convert_grocery_csv --input "Grocery_Inventory_new_v1.csv" --out-dir data --lead-time 7
else:
    print('Skipping conversion; no CSV found in repo root')

# Choose model
USE_PROPHET = False  # set True to use Prophet (slower, better forecasts)
model_name = 'prophet' if USE_PROPHET else 'lstm_stub'

# Run pipeline
!python -m scripts.train_and_update --data-dir data --output-dir outputs --model $model_name --horizon 30 --plot-examples 3

# Return to the parent if we changed directory
if 'google.colab' in sys.modules:
    %cd -


In [ ]:
# Display Key Outputs

import glob
from IPython.display import display

reco = Path('outputs/reorder_recommendations.csv')
if reco.exists():
    df_reco = pd.read_csv(reco)
    display(df_reco.head(20))
else:
    print('No reorder_recommendations.csv found yet')

plots_dir = Path('outputs/plots')
if plots_dir.exists():
    pngs = sorted(glob.glob(str(plots_dir / '*.png')))
    print(f'Found {len(pngs)} plots')
    for p in pngs[:5]:
        display(p)
else:
    print('No plots directory found')


In [ ]:
# Pull Latest Code From GitHub (for VS Code ↔ Colab loop)

if 'google.colab' in sys.modules:
    WORKDIR = '/content/Inventory-Management-Forecasting-using-Machine-Learning'
    %cd $WORKDIR
    !git pull --rebase --autostash
    %cd -
else:
    print('Not in Colab; git pull not needed here')


In [ ]:
# Start Remote Jupyter Server in Colab + Secure Tunnel for VS Code

This will start a Jupyter Server on port 9000 and expose it via Cloudflared.

import secrets
import subprocess
import shlex

if 'google.colab' in sys.modules:
    # Install cloudflared if missing (already installed earlier, but ensure again)
    %pip -q install cloudflared > /dev/null
    # Random token
    TOKEN = secrets.token_hex(16)
    PORT = 9000
    print('Starting Jupyter Server on port', PORT)
    server_cmd = f"jupyter server --ServerApp.token={TOKEN} --ServerApp.allow_origin='*' --no-browser --port={PORT} --NotebookApp.allow_origin='*' --ServerApp.allow_remote_access=True"
    server_proc = subprocess.Popen(shlex.split(server_cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    time.sleep(3)
    print('Starting Cloudflared tunnel...')
    tunnel_proc = subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}', '--no-autoupdate'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    public_url = None
    t0 = time.time()
    while time.time() - t0 < 60 and public_url is None:
        line = tunnel_proc.stdout.readline().strip() if tunnel_proc.stdout else ''
        if line:
            print(line)
        m = re.search(r'https:\/\/[-a-z0-9]+\.trycloudflare\.com', line)
        if m:
            public_url = m.group(0)
            break
    if public_url:
        print('Public URL:', public_url)
        print('Token:', TOKEN)
    else:
        print('Failed to retrieve public URL; check tunnel logs above')
else:
    print('Not running in Colab; skip remote server')


In [ ]:
# Print VS Code Jupyter Server URI and Token

if 'google.colab' in sys.modules:
    try:
        uri = f"{public_url}?token={TOKEN}" if 'public_url' in globals() and public_url else None
        if uri:
            print('Copy this into VS Code:')
            print('Jupyter Server URI:', uri)
            print('Or paste public URL into the Jupyter: Specify Jupyter Server dialog.')
        else:
            print('Public URL not available; please re-run the previous cell to establish the tunnel.')
    except Exception as e:
        print('Error printing URI:', e)
else:
    print('Not in Colab')


# VS Code Usage Instructions

1. In VS Code, open your local repository folder.
2. Install the Jupyter extension if you haven't.
3. Command Palette → "Jupyter: Specify Jupyter Server for connections" → "Existing".
4. Paste the URI printed above (it ends with trycloudflare.com and includes ?token=...).
5. Open any local .ipynb and run cells; execution will happen on the Colab kernel.
6. Edit Python files locally; commit/push. In this notebook, run the "Pull Latest Code" cell to sync.


In [ ]:
# Optional: Lightweight Tests for Cleaning Functions

def _assert(name, cond):
    if not cond:
        raise AssertionError(f'Test failed: {name}')
    print('PASS', name)

# Tests
_assert('to_money $53.82', abs(to_money('$53.82') - 53.82) < 1e-6)
_assert('to_percent 089%', abs(to_percent('089%') - 0.89) < 1e-6)
_assert('to_percent -033%', abs(to_percent('-033%') + 0.33) < 1e-6)
_assert('to_date_safe 1/2/2025', str(to_date_safe('1/2/2025').date()) == '2025-01-02')
print('All cleaning tests passed')
